In [1]:
!pip install python-Levenshtein
!pip install evaluate
!pip install jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.4/3.4 MB 27.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 21.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.0/40.0 MB 34.9 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.5/746.5 kB 7.3 MB/s eta 0:00:00
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2025.3.0
    Uninstalling fsspec-2025.3.0:
      Successfully uninstalled fsspec-2025.3.0
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.4.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.


In [2]:
from evaluate import load
from Levenshtein import distance as levenshtein_distance
from difflib import SequenceMatcher
# import Levenshtein

cer_metric = load("cer")
wer_metric = load("wer")

def compute_metrics(predicted_text, reference_text):
    """
    Compute various text similarity metrics between predicted and reference text.

    Args:
        predicted_text (str or list): The predicted text(s)
        reference_text (str or list): The reference/ground truth text(s)

    Returns:
        dict: Dictionary containing metrics
    """

    # Convert single strings to lists
    if isinstance(predicted_text, str):
        predicted_text = [predicted_text]
    if isinstance(reference_text, str):
        reference_text = [reference_text]

    # Compute CER and WER
    cer_score = cer_metric.compute(predictions=predicted_text, references=reference_text)
    wer_score = wer_metric.compute(predictions=predicted_text, references=reference_text)

    # Levenshtein distance
    lev_dist = levenshtein_distance(predicted_text[0], reference_text[0])
    # similarity = Levenshtein.ratio(predicted_text[0], reference_text[0])

    # Similarity score (0-1)
    seq_matcher = SequenceMatcher(None, predicted_text[0], reference_text[0])
    similarity = seq_matcher.ratio()

    return {
        "cer": cer_score,
        "wer": wer_score,
        "levenshtein_distance": lev_dist,
        "similarity_score": similarity,
    }



/home/scs_deal_learning/miniconda_env/envs/deeplearn_env/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
reference = 'matar'


# predicted = 'matar al enemigs que debia precaverfé, aurique le halle inerme, indefensa'
predicted = 'matar'

result = compute_metrics(predicted,reference)
print(result)

{'cer': 0.0, 'wer': 0.0, 'levenshtein_distance': 0, 'similarity_score': 1.0}


In [4]:
from evaluate import load
from Levenshtein import distance as levenshtein_distance
from difflib import SequenceMatcher

cer_metric = load("cer")
wer_metric = load("wer")

def compute_metrics(predicted_text, reference_text):
    """
    Compute various text similarity metrics between predicted and reference text.

    Args:
        predicted_text (str or list): The predicted text(s)
        reference_text (str or list): The reference/ground truth text(s)

    Returns:
        dict: Dictionary containing metrics
    """

    if isinstance(predicted_text, str):
        predicted_text = [predicted_text]
    if isinstance(reference_text, str):
        reference_text = [reference_text]

    # Compute CER and WER
    cer_score = cer_metric.compute(predictions=predicted_text, references=reference_text)
    wer_score = wer_metric.compute(predictions=predicted_text, references=reference_text)

    levenshtein_distances = []
    similarity_scores = []
    for pred, ref in zip(predicted_text, reference_text):

        # Levenshtein distance
        lev_dist = levenshtein_distance(pred, ref)
        levenshtein_distances.append(lev_dist)

        # Similarity score (0-1)
        seq_matcher = SequenceMatcher(None, pred, ref)
        similarity = seq_matcher.ratio()
        similarity_scores.append(similarity)

    # Calculate averages if multiple samples
    avg_lev_dist = sum(levenshtein_distances) / len(levenshtein_distances)
    avg_similarity = sum(similarity_scores) / len(similarity_scores)

    return {
        "cer": cer_score,
        "wer": wer_score,
        "levenshtein_distance": avg_lev_dist,
        "similarity_score": avg_similarity,
        "all_levenshtein_distances": levenshtein_distances,
        "all_similarity_scores": similarity_scores
    }

In [5]:
reference = ['matar',
             'ô']

# predicted = ['matar al enemigs que debia precaverfé, aurique le halle inerme, indefensa',
#              'ô fe le acometa feguto, è por por la elpalda, ô con arma de luego,ô con ve-']

predicted = ['',
             'ô']

result = compute_metrics(predicted,reference)
print(result)

{'cer': 0.8333333333333334, 'wer': 0.5, 'levenshtein_distance': 2.5, 'similarity_score': 0.5, 'all_levenshtein_distances': [5, 0], 'all_similarity_scores': [0.0, 1.0]}


In [6]:
import json

def evaluate_submission(submission_json_path, label_json_path):
    """
    Evaluates the actual submission files against the corresponding label files.

    The submission file is a JSON file with the following structure:
    {"file_path": [xxx, yyy, ...], "prediction": [xxx, yyy, ...]}
    'file_path' represents the file paths of the data in the target dataset.
    'prediction' represents the output strings from the participant's model.

    The label file is also a JSON file with the following structure:
    {"file_path": [xxx, yyy, ...], "label": [xxx, yyy, ...]}
    'file_path' represents the file paths of the data in the target dataset.
    'label' represents the ground truth strings for the data.
    """
    with open(submission_json_path, 'r') as f:
        submission = json.load(f)

    with open(label_json_path, 'r') as f:
        labels = json.load(f)

    # Create dictionaries for easier lookup by file_path
    submission_dict = dict(zip(submission['file_path'], submission['prediction']))
    labels_dict = dict(zip(labels['file_path'], labels['label']))

    # Align predictions and references based on file_path
    predictions = []
    references = []
    for file_path in labels['file_path']:
        if file_path in submission_dict:
            predictions.append(submission_dict[file_path].lower())
            references.append(labels_dict[file_path].lower())
        else:
            # Handle cases where a file_path in labels is not in submission
            print(f"Warning: File path '{file_path}' not found in submission file.")
            predictions.append("")
            references.append(labels_dict[file_path])


    # Compute metrics using the existing compute_metrics function
    results = compute_metrics(predictions, references)

    return results

In [7]:
label = {
    'file_path' : ['book1_page1','book1_page2','book2_page1'],
    'label' : [
        "SI POR evitar UN pecado mortal",
        "aveys de poner vuestra vida en pe",
        "ligro arriesgalda QUE es el mejor",
    ]
}

label_object = json.dumps(label, indent=4)
save_path = 'label.json'
with open(save_path, 'w') as f:
    f.write(label_object)

submission = {
    'file_path' : ['book1_page1','book1_page2','book2_page1'],
    'prediction' : [
        "Si por euitar n pecado mortal",
        "aueys de poner uuestra uida en pe",
        "ligro arriesgalda que es el mejor",
    ]

}

submission_object = json.dumps(submission, indent=4)
save_path = 'submission.json'
with open(save_path, 'w') as f:
    f.write(submission_object)


label_json_path = 'label.json'
submission_json_path = 'submission.json'

results = evaluate_submission(submission_json_path, label_json_path)
print(results)

{'cer': 0.052083333333333336, 'wer': 0.2631578947368421, 'levenshtein_distance': 1.6666666666666667, 'similarity_score': 0.9527478171545969, 'all_levenshtein_distances': [2, 3, 0], 'all_similarity_scores': [0.9491525423728814, 0.9090909090909091, 1.0]}


In [8]:
# When missing entry in submission file
# Ground truth (label.json)
label = {
    "file_path": [
        "book1_page1",
        "book1_page2",
        "book1_page3",
        "book1_page4",
        "book1_page5",
        "book1_page6",
        "book1_page7"
    ],
    "label": [
        "Si por evitar un pecado mortal",
        "aveys de poner vuestra vida en pe",
        "ligro arriesgalda que es el mejor",
        "empleo que della podeys hazer y",
        "de vuestra hazienda para este fin en",
        "redemir cautivos y sacar mugeres",
        "de pecado dotandolas liberalmen"
    ]
}

submission = {
    "file_path": [
        "book1_page1",
        "book1_page2",
        "book1_page3",
        "book1_page5",
        "book1_page6",
        "book1_page7"
    ],
    "prediction": [
        "si por euitar n pecado mortal",
        "aueys de poner uuestra uida en pe",
        "ligro arriesgalda que es el mejor",
        "hazienda para este fin en",
        "redemir cautiuos y sacar mugeres",
        "de pecado dotandolas liberalmen"
    ]
}

with open('label1.json', 'w') as f:
    json.dump(label, f, indent=4)

with open('submission1.json', 'w') as f:
    json.dump(submission, f, indent=4)


In [9]:
label_json_path = 'label1.json'
submission_json_path = 'submission1.json'

results = evaluate_submission(submission_json_path, label_json_path)
print(results)

{'cer': 0.21238938053097345, 'wer': 0.34146341463414637, 'levenshtein_distance': 6.857142857142857, 'similarity_score': 0.8066665118016189, 'all_levenshtein_distances': [2, 3, 0, 31, 11, 1, 0], 'all_similarity_scores': [0.9491525423728814, 0.9090909090909091, 1.0, 0.0, 0.819672131147541, 0.96875, 1.0]}
